# Kagge Basic LSTM Using Stroke Data


In [1]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
import cv2
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model
from random import randint
import pandas as pd
import pickle
import ast
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
# from keras.models import load_model

Using TensorFlow backend.


In [2]:
f = open("/data/miniclasses.txt","r")
classes = f.readlines()
f.close()
classes = [c.replace('\n','').replace(' ','_') for c in classes]
n = len(classes)
print('no', n)
vals = [x for x in range(0,n)]
classmap = dict(zip(classes, vals))
classmaprev = dict(zip(vals,classes))

no 340


In [3]:
all_files = glob.glob(os.path.join('shuffledtrain2/', '*.csv'))
batchsize =  512
img_size = 32
N_CLASSES = 340
STROKE_COUNT = 196
STEPS = 800
EPOCHS = 100
size = 32
TEST_DIR = ''
N_FILES = 100

In [4]:
print(all_files)
t = pd.read_csv(all_files[1], nrows = 10)
t['drawing'] = t['drawing'].apply(ast.literal_eval)
labels = t['word']
labels = [c.replace(' ','_') for c in labels]
idx = [classmap[x] for x in labels]
print(idx)

['shuffledtrain2/shuffledtrain2_56.csv', 'shuffledtrain2/shuffledtrain2_41.csv', 'shuffledtrain2/shuffledtrain2_76.csv', 'shuffledtrain2/shuffledtrain2_49.csv', 'shuffledtrain2/shuffledtrain2_99.csv', 'shuffledtrain2/shuffledtrain2_18.csv', 'shuffledtrain2/shuffledtrain2_97.csv', 'shuffledtrain2/shuffledtrain2_9.csv', 'shuffledtrain2/shuffledtrain2_79.csv', 'shuffledtrain2/shuffledtrain2_4.csv', 'shuffledtrain2/shuffledtrain2_27.csv', 'shuffledtrain2/shuffledtrain2_8.csv', 'shuffledtrain2/shuffledtrain2_85.csv', 'shuffledtrain2/shuffledtrain2_93.csv', 'shuffledtrain2/shuffledtrain2_13.csv', 'shuffledtrain2/shuffledtrain2_86.csv', 'shuffledtrain2/shuffledtrain2_100.csv', 'shuffledtrain2/shuffledtrain2_91.csv', 'shuffledtrain2/shuffledtrain2_59.csv', 'shuffledtrain2/shuffledtrain2_24.csv', 'shuffledtrain2/shuffledtrain2_94.csv', 'shuffledtrain2/shuffledtrain2_3.csv', 'shuffledtrain2/shuffledtrain2_10.csv', 'shuffledtrain2/shuffledtrain2_62.csv', 'shuffledtrain2/shuffledtrain2_47.csv', 's

In [5]:
def draw_cv2(raw_strokes, size=256, lw=6, time_color=True):
    img = np.zeros((256, 256), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    if size != 256:
        return cv2.resize(img, (size, size))
    else:
        return img
    

def _stack_it(raw_strokes):
    """preprocess the string and make 
    a standard Nx3 stroke vector"""
    stroke_vec = ast.literal_eval(raw_strokes) # string->list
    # unwrap the list
    in_strokes = [(xi,yi,i)  
     for i,(x,y) in enumerate(stroke_vec) 
     for xi,yi in zip(x,y)]
    c_strokes = np.stack(in_strokes)
    # replace stroke id with 1 for continue, 2 for new
    c_strokes[:,2] = [1]+np.diff(c_strokes[:,2]).tolist()
    c_strokes[:,2] += 1 # since 0 is no stroke
    # pad the strokes with zeros
    return pad_sequences(c_strokes.swapaxes(0, 1), 
                         maxlen=STROKE_COUNT, 
                         padding='post').swapaxes(0, 1)
    
def image_generator_xd(all_files,classmap, size, batchsize,ks,N_CLASSES, lw=6, time_color=True):
    while True:
        for k in np.random.permutation(ks):
            filename = all_files[k]
            for df in pd.read_csv(filename, chunksize=batchsize):
                y = np.empty([0])
                df['drawing'] = df['drawing'].map(_stack_it)
                labels = df['word']
                labels = [c.replace(' ','_') for c in labels]
                idx = [classmap[x] for x in labels]
                y = np.append(y, idx)
                x = np.stack(df['drawing'], 0)
                #x = preprocess_input(x).astype(np.float32)
                y = keras.utils.to_categorical(y, N_CLASSES)
                yield x, y

def df_to_image_array_xd(df, size, lw=6, time_color=True):
    df['drawing'] = df['drawing'].apply(ast.literal_eval)
    x = np.zeros((len(df), size, size, 1))
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw, time_color=time_color)
    x = preprocess_input(x).astype(np.float32)
    return x

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def get_available_gpus():
    from tensorflow.python.client import device_lib
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

def preds2catids(predictions):
    return pd.DataFrame(np.argsort(-predictions, axis=1)[:, :3], columns=['a', 'b', 'c'])

In [6]:
train_datagen = image_generator_xd(all_files,classmap,size=img_size, N_CLASSES= N_CLASSES,batchsize=batchsize,ks=range(N_FILES))

In [7]:
x_, y= next(train_datagen)
print(x_.shape)
print(y.shape)

(512, 196, 3)
(512, 340)


In [9]:
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv1D, LSTM, Dense, Dropout, Bidirectional
if len(get_available_gpus())>0:
    from keras.layers import CuDNNLSTM as LSTM # this one is about 3x faster on GPU instances
stroke_read_model = Sequential()
stroke_read_model.add(BatchNormalization(input_shape = (None,)+x_.shape[2:]))
# filter count and length are taken from the script https://github.com/tensorflow/models/blob/master/tutorials/rnn/quickdraw/train_model.py
stroke_read_model.add(Conv1D(48, (5,)))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(Conv1D(64, (5,)))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(Conv1D(96, (3,)))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(LSTM(128, return_sequences = True))
stroke_read_model.add(LSTM(128, return_sequences = False))
stroke_read_model.add(Dense(512))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(Dense(340, activation = 'softmax'))
stroke_read_model.compile(optimizer = 'adam', 
                          loss = 'categorical_crossentropy', 
                          metrics = ['categorical_accuracy', top_3_accuracy])
stroke_read_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, None, 3)           12        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 48)          768       
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 48)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 64)          15424     
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 64)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 96)          18528     
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 96)          0         
__________

In [ ]:
from tensorflow.keras.metrics import top_k_categorical_accuracy
def top_3_accuracy(x,y): 
    t3 = top_k_categorical_accuracy(x,y, 3)
    return t3


stroke_read_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', top_3_accuracy])

hists = []
hist = stroke_read_model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
)
hists.append(hist)

Epoch 1/100
800/800 [==============================] - 95s 118ms/step - loss: 5.6602 - acc: 0.0105 - top_3_accuracy: 0.0287
Epoch 2/100
800/800 [==============================] - 94s 118ms/step - loss: 5.3318 - acc: 0.0217 - top_3_accuracy: 0.0536
Epoch 3/100
800/800 [==============================] - 94s 118ms/step - loss: 5.0162 - acc: 0.0388 - top_3_accuracy: 0.0916
Epoch 4/100
800/800 [==============================] - 95s 119ms/step - loss: 4.6837 - acc: 0.0649 - top_3_accuracy: 0.1463
Epoch 5/100
800/800 [==============================] - 94s 118ms/step - loss: 4.2828 - acc: 0.1067 - top_3_accuracy: 0.2239
Epoch 6/100
800/800 [==============================] - 96s 120ms/step - loss: 3.7497 - acc: 0.1773 - top_3_accuracy: 0.3393
Epoch 7/100
620/800 [======================>.......] - ETA: 20s - loss: 3.3785 - acc: 0.2404 - top_3_accuracy: 0.4264

# Submission

In [ ]:
sub_df = pd.read_csv(test_path)
sub_df['drawing'] = sub_df['drawing'].map(_stack_it)
sub_vec = np.stack(sub_df['drawing'].values, 0)
sub_pred = stroke_read_model.predict(sub_vec, verbose=True, batch_size=batch_size)
top_3_pred = [word_encoder.classes_[np.argsort(-1*c_pred)[:3]] for c_pred in sub_pred]
top_3_pred = [' '.join([col.replace(' ', '_') for col in row]) for row in top_3_pred]
sub_df['word'] = top_3_pred
sub_df[['key_id', 'word']].to_csv('submission.csv', index=False)